In [2]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from konlpy.tag import Kkma
from selenium import webdriver
from transformers import pipeline
from datetime import datetime, timedelta
from wordcloud import WordCloud, STOPWORDS

c:\Users\icn97\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##  연수 후기 크롤링

* 크롬을 통해 Tsherpa 연수 후기 사이트로 들어가기

In [ ]:
driver = webdriver.Chrome()

tsherpa_url = "https://edu.tsherpa.co.kr/LectureInfo/LectureReviews2"
driver.get(tsherpa_url)

* Tsherpa에 모든 후기 크롤링

In [ ]:
ind = 1  # 현재 복사한 순서
no = 1 # 1~10페이지 중 위치한 곳
page = 1  # 현재 페이지 번호
list1 = []  # 결과물이 저장되는 리스트


while 1:
    try:
        driver.switch_to.window(driver.window_handles[0])  
        
        driver.find_element(by='xpath',
                    value = f'//*[@id="mainForm"]/table/tbody/tr[{ind}]/td[2]/a').click()

        time.sleep(0.2)

        driver.switch_to.window(driver.window_handles[1]) 
        
        name = driver.find_element(by='xpath',
               value = '//*[@id="mainForm"]/div[1]/span/a').text

        title = driver.find_element(by='xpath', value = '//*[@id="mainForm"]/div[2]/table/tbody/tr[1]/td').text

        star= (driver.find_element(by='xpath',value = f'//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[1]/img')).get_attribute('alt')

        contents = driver.find_element(by='xpath',
                           value = '//*[@id="mainForm"]/div[2]/table/tbody/tr[3]/td').text
        
        date = driver.find_element(by='xpath',
                   value = '//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[2]').text.split(' ')[3]


        driver.find_element(by = 'xpath',
                           value = '//*[@id="mainForm"]/div[2]/div/a').click()

        driver.switch_to.window(driver.window_handles[0])  
        
        
        list1.append([name, title, star, contents, date])
        
        ind += 1
       

    except:
        no += 1
        if no < 11:
            driver.find_element(by='xpath',value = f'//*[@id="mainForm"]/div[3]/a[{no+1}]').click()
            ind = 1
            page += 1
            continue


        elif no == 11:
            driver.find_element(by='xpath',value = '//*[@id="mainForm"]/div[3]/a[12]/img').click()
            no = 1
            ind = 1
            page += 1
            continue
            
            
            
        else:
            break


df = pd.DataFrame(list1)
df.rename(columns = { 0 : 'name', 1 : 'title', 2 : 'star', 3 : 'comments', 4 : 'date'},inplace = True)
df = df.drop_duplicates(['contents'])
df.to_csv('tsherpa.csv',index = False)

In [19]:
tsherpa = pd.read_csv('./tsherpa.csv')

* 크롬을 통해 icecream 연수 후기 사이트로 들어가기

In [ ]:
driver = webdriver.Chrome()

review_search_url = "https://teacher.i-scream.co.kr/help/afterword/list.do?sso=ok"
driver.get(review_search_url)

* icecream 연수 후기가 tsherpa에 비해서 매우 많기 때문에 데이터 수를 조절하기 위해 한 페이지에서 1개씩만 크롤링

In [ ]:
list1 = []


while 1:
    try:
        for j in range(3, 13): # 한 페이지 넘어갈 때 10개씩 있음(1번은 이전10페이지로)
            driver.find_element(by='xpath',value = f'//*[@id="content"]/div[3]/table/tbody/tr[1]/td[3]/a').click() # 연수후기 페이지 클릭

            name = driver.find_element(by='xpath',value = f'//*[@id="bbs_wrap"]/div/dl[2]/dd/a').text # 연수명
            title = driver.find_element(by='xpath',value = f'//*[@id="bbs_wrap"]/div/dl[1]/dd').text # 연수후기 제목
            el_rated = driver.find_element(by='xpath',value = f'//*[@id="bbs_wrap"]/div/dl[3]/dd[1]/span/span')
            rated = el_rated.get_attribute('class')  # 별점
            review = driver.find_element(by='xpath',value = f'//*[@id="bbs_wrap"]/div/div[1]').text # 내용
            classified = driver.find_element(by='xpath',value = f'//*[@id="bbs_wrap"]/div/dl[3]/dd[2]').text  # 분류
            date = driver.find_element(by='xpath',value = f'//*[@id="bbs_wrap"]/div/dl[4]/dd[2]').text  # 날짜




            list1.append([name, title, rated, review, classified, date])
            time.sleep(0.1)
            driver.find_element(by='xpath',value = f'//*[@id="btn-list"]').click() # 목록으로 돌아가기
            time.sleep(0.2)
            driver.find_element(by='xpath',value = f'//*[@id="paging"]/ul/li[{j}]/a').click() # 옆 페이지 넘어가기 ex) 1->2
            
            df = pd.DataFrame(list1)
            df.to_csv('icecream.csv')
            time.sleep(0.1)
        
    except:
        break

In [28]:
icecream = pd.read_csv('./icecream.csv')

## 감정분석

In [16]:
def word():
    x = input()
    classifier = pipeline("text-classification", model="matthewburke/korean_sentiment")
    custom_tweet = f'b{x}'
    preds = classifier(custom_tweet, return_all_scores=True)
    is_positive = preds[0][1]['score'] > 0.5
    print(is_positive)

* Tsherpa의 후기 중 title과 comments를 같이 감정분석하기 위해서 title_comments 변수로 추가

In [14]:
tsherpa['title_comments'] = tsherpa['title'] + ' ' + tsherpa['comments']

* 감정분석을 하기 위해서 글자수가 512개 이상인 행은 삭제

In [15]:
tsherpa = tsherpa[tsherpa['title_comments'].apply(lambda x: False if len(x) > 512 else True)]

* Tsherpa에 대해서 감정 분석 진행

In [ ]:
list_positive = []

for i in tsherpa.loc[:,'comments']:
    preds = classifier(i, return_all_scores=True)
    is_positive = preds[0][1]['score'] > 0.5
    list_positive.append(is_positive)

In [ ]:
tsherpa['emotion'] = list_positive

In [ ]:
tsherpa.to_csv('tsherpa.csv',index=False)

In [ ]:
tsherpa = pd.read_csv('./tsherpa.csv')

* icecream의 후기 중에서 title과 comments를 같이 감정분석하기 위해서 title_comments 변수로 추가

In [ ]:
icecream['title_comments'] = icecream['title'] + ' ' + icecream['comments']

* 감정분석을 하기 위해서 글자수가 512개 이상인 행은 삭제

In [ ]:
icecream = icecream[icecream['title_comments'].apply(lambda x: False if len(x) > 512 else True)]

* icecream에 대해서 감정분석 진행

In [ ]:
list_positive = []

for i in icecream.loc[:,'comments']:
    preds = classifier(i, return_all_scores=True)
    is_positive = preds[0][1]['score'] > 0.5
    list_positive.append(is_positive)

In [ ]:
icecream['emotion'] = list_positive

In [ ]:
icecream.to_csv('icecream.csv',index=False)

In [ ]:
icecream = pd.read_csv('./icecream.csv')

In [ ]:
icecream

## tokeninzing

In [ ]:
kkma = Kkma()

* 완전한 한글을 제외한 문자 제거

In [ ]:
tsherpa['title_comments'] = tsherpa['title_comments'].replace('[^가-힣]', ' ', regex = True)
icecream['title_comments'] = icecream['title_comments'].replace('[^가-힣]', ' ', regex = True)

#### tsherpa

* 감정분류가 긍정으로 된 후기를 단어로 구분

In [5]:
tsherpa_true = tsherpa[tsherpa['emotion_analysis'] == True]
tsherpa_nouns_true = tsherpa_true['title_comments'].apply(kkma.nouns)

* 감정분류가 부정으로 된 후기를 단어로 구분

In [ ]:
tsherpa_false = tsherpa[tsherpa['emotion_analysis'] == False]
tsherpa_nouns_false = tsherpa_false['title_comments'].apply(kkma.nouns)

* 긍정으로 분류된 후기들을 구성하는 단어를 빈도수로 정렬

In [ ]:
tsherpa_nouns_true = tsherpa_nouns_true.explode()
tsherpa_words_true = pd.DataFrame({'word' : tsherpa_nouns_true})
tsherpa_words_true['count'] = tsherpa_words_true['word'].str.len()
tsherpa_words_true = tsherpa_words_true.query('count>1')
tsherpa_true = tsherpa_words_true.groupby('word',as_index = False).count().sort_values('count',ascending = False)

* 부정으로 분류된 후기들을 구성하는 단어를 빈도수로 정렬

In [ ]:
tsherpa_nouns_false = tsherpa_nouns_false.explode()
tsherpa_words_false = pd.DataFrame({'word' : tsherpa_nouns_false})
tsherpa_words_false['count'] = tsherpa_words_false['word'].str.len()
tsherpa_words_false = tsherpa_words_false.query('count>1')
tsherpa_false = tsherpa_words_false.groupby('word',as_index = False).count().sort_values('count',ascending = False)

#### icecream

* 감정분류가 긍정으로 된 후기를 단어로 구분

In [ ]:
icecream_true = icecream[icecream['emotion'] == True]
icecream_nouns_true = icecream_true['title_comments'].apply(kkma.nouns)

* 감정분류가 부정으로 된 리뷰를 단어로 구분

In [ ]:
icecream_false = icecream[icecream['emotion'] == False]
icecream_nouns_false = icecream_false['title_comments'].apply(kkma.nouns)

* 긍정으로 분류된 리뷰들을 구성하는 단어를 빈도수로 정렬

In [ ]:
icecream_nouns_true = icecream_nouns_true.explode()
icecream_words_true = pd.DataFrame({'word' : icecream_nouns_true})
icecream_words_true['count'] = icecream_words_true['word'].str.len()
icecream_words_true = icecream_words_true.query('count>1')
icecream_words_true = icecream_words_true.groupby('word',as_index = False).count().sort_values('count',ascending = False)
icecream_words_true.reset_index(drop = True, inplace = True)

* 부정으로 분류된 리뷰들을 구성하는 단어를 빈도수로 정렬

In [ ]:
icecream_nouns_false = icecream_nouns_false.explode()
icecream_words_false = pd.DataFrame({'word' : icecream_nouns_false})
icecream_words_false['count'] = icecream_words_false['word'].str.len()
icecream_words_false = icecream_words_false.query('count>1')
icecream_words_false = icecream_words_false.groupby('word',as_index = False).count().sort_values('count',ascending = False)
icecream_words_false.reset_index(drop = True, inplace = True)

## 워드클라우드
* 불용어 제거를 위해 긍정후기와 부정후기에서 공통적으로 보이는 단어들을 각 후기에서 제거
* 워드클라우드 그리기

#### tsherpa

* 긍정후기와 부정후기에서 공통적으로 보이는 단어 추출

In [ ]:
tsherpa_inner = pd.merge(tsherpa_true,tsherpa_false,how='inner', on = 'word')
tsherpa_inner = tsherpa_inner['word'].head(20)

* 긍정후기에서 공통단어 제거

In [ ]:
tsherpa_true.index = tsherpa_true['word']

In [ ]:
for i in range(0,len(tsherpa_inner)):
    tsherpa_true.drop([tsherpa_inner[i]],inplace = True)

* 부정후기에서 공통단어 제거

In [ ]:
tsherpa_false.index = tsherpa_false['word']

In [ ]:
for i in range(0,len(tsherpa_inner)):
    tsherpa_false.drop([tsherpa_inner[i]],inplace = True)

* 긍정 워드클라우드

In [ ]:
tsherpa_true = tsherpa_true.set_index('word').to_dict()['count']
im = Image.open('heart.png')
mask = np.array(im)
wordcloud = WordCloud(
font_path = "malgun",
    width = 400,
    height = 400,
    mask = mask,
    max_font_size = 100,
    background_color='white',
    colormap = 'autumn'
).generate_from_frequencies(tsherpa_true)
plt.figure()
plt.imshow(wordcloud)
plt.axis('off')

* 부정 워드클라우드

In [ ]:
tsherpa_false = tsherpa_false.set_index('word').to_dict()['count']
im = Image.open('heart.png')
mask = np.array(im)
wordcloud = WordCloud(
font_path = "malgun",width = 400,height = 400,mask = mask,max_font_size = 100, background_color='white',   stopwords = stopwords
).generate_from_frequencies(tsherpa_false)
plt.figure()
plt.imshow(wordcloud)
plt.axis('off')

#### icecream

* 긍정후기와 부정후기에서 공통적으로 보이는 단어 추출

In [ ]:
icecream_inner = pd.merge(icecream_true,icecream_false,how='inner', on = 'word')
icecream_inner = icecream_inner['word'].head(20)

* 긍정후기에서 공통단어 제거

In [ ]:
icecream_true.index = icecream_true['word']

In [ ]:
for i in range(0,len(icecream_inner)):
    icecream_true.drop([icecream_inner[i]],inplace = True)

* 부정후기에서 공통단어 제거

In [ ]:
icecream_false.index = icecream_false['word']

In [ ]:
for i in range(0,len(icecream_inner)):
    icecream_false.drop([icecream_inner[i]],inplace = True)

* 긍정 워드클라우드

In [ ]:
icecream_true = icecream_true.set_index('word').to_dict()['count']
im = Image.open('heart.png')
mask = np.array(im)
wordcloud = WordCloud(
font_path = "malgun",
    width = 400,
    height = 400,
    mask = mask,
max_font_size = 100, 
background_color='white',
colormap = 'autumn'
).generate_from_frequencies(icecream_true)
plt.figure()
plt.imshow(wordcloud)
plt.axis('off')

* 부정 워드클라우드

In [ ]:
icecream_false = icecream_false.set_index('word').to_dict()['count']
im = Image.open('heart.png')
mask = np.array(im)
wordcloud = WordCloud(
font_path = "malgun",
    width = 400,
    height = 400,
    mask = mask,
max_font_size = 100, 
background_color='white'
).generate_from_frequencies(icecream_false)
plt.figure()
plt.imshow(wordcloud)
plt.axis('off')